In [1]:
import networkx
import numpy as np

from matplotlib import pyplot

import random

%matplotlib inline

In [2]:
grid_graph = networkx.grid_graph([5,5])

In [17]:
def generate_random_cz_schema(lattice_graph):
    loop_state = networkx.graphviews.generic_graph_view(lattice_graph)
    visited_edges = []
    cz_schema = []
    layer_counter = 1
    while not networkx.is_empty(loop_state):
        layer_state = networkx.graphviews.generic_graph_view(loop_state)
        nodes_to_ignore = set()
        edges = []
        while not networkx.is_empty(layer_state):
            random_edge = random.choice(list(layer_state.edges))
            neighbours = [neighbour for neighbour in lattice_graph.adj[random_edge[0]]]
            neighbours += [neighbour for neighbour in lattice_graph.adj[random_edge[1]]]
            for n in neighbours:
                nodes_to_ignore.add(n)
            nodes_to_ignore.add(random_edge[0])
            nodes_to_ignore.add(random_edge[1])
            edges.append(random_edge)
            layer_state = networkx.graphviews.subgraph_view(
                loop_state,
                filter_node=networkx.filters.hide_nodes(nodes_to_ignore))
            assert(random_edge not in layer_state.edges)      
        for e in edges:
            visited_edges.append(e)
        cz_schema.append(edges)
        layer_counter += 1
        loop_state = networkx.graphviews.subgraph_view(
            lattice_graph,
            filter_edge=networkx.filters.hide_edges(visited_edges))
    return cz_schema


In [28]:
grid_graph.nodes

NodeView(((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)))

In [35]:
def index_to_coords(index, dimensions):
    def div_n_mod(val, div, mod):
        return (val//div)%mod
    divs = [1]
    for d in dimensions:
        divs.append(divs[-1]*d)
    return tuple(div_n_mod(index, div, mod) for div, mod in zip(divs, dimensions))
        
def coords_to_index(coords, dimensions):
    divs = [1]
    for d in dimensions:
        divs.append(divs[-1]*d)
    index = 0
    for coord, multiplies in zip(coords, divs[:-1]):
        index += coord*multiplies
    return index

In [38]:
print(index_to_coords(10, (3,4)))
print(coords_to_index((1,3), (3,4)))

(1, 3)
10


In [50]:
len(grid_graph.edges)

40

In [53]:
loop_state = networkx.graphviews.generic_graph_view(grid_graph)
visited_edges = []
cz_schema = []
layer_counter = 0
top_node = list(grid_graph.nodes)[-1]
dimension = len(top_node)
lengths = [dim+1 for dim in top_node]
visited_edges = []
while not networkx.is_empty(loop_state):
    layer_state = networkx.graphviews.generic_graph_view(loop_state)
    excluded_nodes = set()
    main_axis = layer_counter % dimension
    points = 1
    alt_axes = [i for i in range(dimension) if i != main_axis]
    alt_lengths = [lengths[i] for i in range(dimension) if i != main_axis]
    for axis in alt_axes:
        points *= top_node[axis]
    nodes_to_ignore = set()
    edges = []
    for p in range(points):
        if networkx.is_empty(layer_state):
            break
        position = index_to_coords(p, alt_lengths)
        for i in range(lengths[main_axis] - 1):
            node = position[:main_axis] + tuple([i]) + position[main_axis:]
            if node in layer_state.nodes:
                node_2 = position[:main_axis] + tuple([i+1]) + position[main_axis:]
                edge = tuple((node, node_2))
                if edge in layer_state.edges:
                    neighbours = [neighbour for neighbour in grid_graph.adj[node]]
                    neighbours += [neighbour for neighbour in grid_graph.adj[node_2]]
                    for n in neighbours:
                        nodes_to_ignore.add(n)
                    excluded_nodes.add(node)
                    excluded_nodes.add(node_2)
                    edges.append(edge)
                    layer_state = networkx.graphviews.subgraph_view(
                        loop_state,
                        filter_node=networkx.filters.hide_nodes(nodes_to_ignore))
    
    for e in edges:
        visited_edges.append(e)
    print(len(visited_edges))
    layer_counter += 1
    cz_schema.append(edges)
    loop_state = networkx.graphviews.subgraph_view(
            grid_graph,
            filter_edge=networkx.filters.hide_edges(visited_edges))
    print(loop_state.edges)


4
[((0, 0), (0, 1)), ((0, 1), (1, 1)), ((0, 1), (0, 2)), ((0, 2), (0, 3)), ((0, 3), (1, 3)), ((0, 3), (0, 4)), ((0, 4), (1, 4)), ((1, 0), (2, 0)), ((1, 0), (1, 1)), ((1, 1), (2, 1)), ((1, 1), (1, 2)), ((1, 2), (2, 2)), ((1, 2), (1, 3)), ((1, 3), (2, 3)), ((1, 3), (1, 4)), ((1, 4), (2, 4)), ((2, 0), (3, 0)), ((2, 0), (2, 1)), ((2, 1), (3, 1)), ((2, 1), (2, 2)), ((2, 2), (3, 2)), ((2, 2), (2, 3)), ((2, 3), (3, 3)), ((2, 3), (2, 4)), ((2, 4), (3, 4)), ((3, 0), (3, 1)), ((3, 1), (4, 1)), ((3, 1), (3, 2)), ((3, 2), (3, 3)), ((3, 3), (4, 3)), ((3, 3), (3, 4)), ((3, 4), (4, 4)), ((4, 0), (4, 1)), ((4, 1), (4, 2)), ((4, 2), (4, 3)), ((4, 3), (4, 4))]
8
[((0, 1), (1, 1)), ((0, 1), (0, 2)), ((0, 2), (0, 3)), ((0, 3), (1, 3)), ((0, 4), (1, 4)), ((1, 0), (2, 0)), ((1, 0), (1, 1)), ((1, 1), (2, 1)), ((1, 1), (1, 2)), ((1, 2), (2, 2)), ((1, 2), (1, 3)), ((1, 3), (2, 3)), ((1, 3), (1, 4)), ((1, 4), (2, 4)), ((2, 0), (3, 0)), ((2, 1), (3, 1)), ((2, 1), (2, 2)), ((2, 2), (3, 2)), ((2, 2), (2, 3)), ((2,

KeyboardInterrupt: 

In [170]:
a = [0,1,2,4,5]
a.insert(3,3)
a

[0, 1, 2, 3, 4, 5]

In [8]:
tuple((1,2))

(1, 2)